In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

In [7]:
response.choices[0].message.content

"I'm sorry, but I need more information to provide a relevant answer. Could you specify which course you are referring to, including details such as the start date, enrollment deadlines, and any prerequisites? If you have this information, I can help determine if it's too late to join."